In [1]:
import pandas as pd
import cv2
import os

In [3]:
images = []
labels = []
categories = ['sport', 'agriculture']  # Replace with your actual categories

for category in categories:  # categories you have like 'sports', 'shopping', etc.
    for image in os.listdir(f'{category}'):
        img = cv2.imread(f'{category}/{image}')
        img = cv2.resize(img, (224, 224))  # Resize if necessary
        images.append(img)
        labels.append(category)

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import numpy as np

labels = np.array(labels)
# y = np.argmax(labels, axis=1)
print(labels)
y= labels


['sport' 'sport' 'sport' ... 'agriculture' 'agriculture' 'agriculture']


In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Normalize image vectors
images = np.array(images) / 255.0

# Split the data
X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2)



In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [7]:

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more layers as necessary

model.add(Flatten())  # Flatten before dense layers
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(categories)))  # Output layer: one per category
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [8]:
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1)


Epoch 1/10


25/25 [==============================] - 17s 611ms/step - loss: 2.4330 - accuracy: 0.7711 - val_loss: 0.4621 - val_accuracy: 0.7558
Epoch 2/10
25/25 [==============================] - 14s 560ms/step - loss: 0.2659 - accuracy: 0.9142 - val_loss: 0.2254 - val_accuracy: 0.8837
Epoch 3/10
25/25 [==============================] - 14s 565ms/step - loss: 0.1956 - accuracy: 0.9272 - val_loss: 0.0967 - val_accuracy: 0.9419
Epoch 4/10
25/25 [==============================] - 14s 556ms/step - loss: 0.1097 - accuracy: 0.9597 - val_loss: 0.0805 - val_accuracy: 0.9651
Epoch 5/10
25/25 [==============================] - 14s 570ms/step - loss: 0.1033 - accuracy: 0.9584 - val_loss: 0.1182 - val_accuracy: 0.9884
Epoch 6/10
25/25 [==============================] - 14s 572ms/step - loss: 0.0866 - accuracy: 0.9727 - val_loss: 0.0838 - val_accuracy: 0.9651
Epoch 7/10
25/25 [==============================] - 14s 579ms/step - loss: 0.0681 - accuracy: 0.9753 - val_loss: 0.0789 - val_accuracy: 0.98

In [9]:
model.evaluate(X_test, y_test)


7/7 [==============================] - 1s 87ms/step - loss: 0.0061 - accuracy: 1.0000


[0.006072469986975193, 1.0]

In [10]:
import requests

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224)) / 255.0
    prediction = model.predict(np.array([img]))
    return label_encoder.inverse_transform([np.argmax(prediction)])[0]

def predict_image_from_url(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, (224, 224)) / 255.0
            prediction = model.predict(np.array([img]))
            return label_encoder.inverse_transform([np.argmax(prediction)])[0]
        else:
            return "Failed to decode image"
    else:
        return "Failed to download image"

# predict_image_from_url('https://location-voitures-essaouira.com/wp-content/uploads/2015/05/location-voitures-essaouira-rent-car-essaouira%20(2).png')

In [14]:
predict_image('./sport/sailboat racing_042.jpg')
#TEST ------------------------------------------------
#predict_image("./agriculture/sugarcane036a.jpeg")

1/1 [==============================] - 0s 29ms/step


'sport'

In [15]:
# import matplotlib. pyplot as plt
# plt.scatter(y_test,y_pred)
# plt.grid


# Assuming 'model' is your trained Keras model
model.save('my_model.h5')


C:\Users\mamad\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 619ms/step


'agriculture'

In [22]:
import numpy as np
classes = np.load('classes.npy')
print(classes)


['sport' 'driving']


In [23]:
import numpy as np

# List of class names
classes = ['sport', 'agriculture']

# Save this array to a .npy file
np.save('classes.npy', classes)